<a id="top"></a>
# Predicting Critical Temperature of Superconductors With a Linear Regression Model Using L1 (LASSO) & L2 (RIDGE) Regularization Techniques
## Contents

* <a href="#Dependencies">Dependencies</a>
* <a href="#Load">Loading the Data</a>
* <a href="#DataPrep">Model Preperation</a>
    * <a href="#Scale">Data Scaling</a>
    * <a href="#TTSplit">Train/Test Split</a>
    * <a href="#Target">Seperate Target</a>
* <a href="#ModelPrep">Model Preperation</a>
* <a href="#Eval">Model Evaluation</a>
* <a href="#ExplFeat">Discuss Important Features</a>
* <a href="#Cite">Citations</a>

----
<a id="Dependencies"></a>
<a href="#top">Back to Top</a>
## Dependencies

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, precision_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, KFold
from scipy.stats import ttest_ind

----
<a id="Load"></a>
<a href="#top">Back to Top</a>
## Loading the Data
    Ensure that your data is clean and properly preprocessed. Handle missing values, encode categorical variables if necessary, and address any outliers.

In [3]:
# file path
file_path1 = "train.csv" # previously cleaned
file_path2 = "unique_m.csv" # previously cleaned

# Load the dataset
train = pd.read_csv(file_path1)
unique = pd.read_csv(file_path2)

# Set the maximum number of columns to display to None
pd.set_option('display.max_columns', None)

In [4]:
train.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.25,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237540,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,3572.596624,81.8375,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.9055,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.20,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,3632.649185,90.8900,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.7844,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,775.425,1010.820000,718.152900,939.009036,1.305967,0.773620,810.6,743.164286,323.811808,354.804183,160.25,104.685714,136.126003,84.214573,1.259244,1.132547,205,49.314286,75.237540,67.797712,4654.35725,2999.159429,724.953211,53.974022,1.033129,0.760305,8958.571,1667.383429,3306.162897,3592.019281,81.8375,112.213571,60.123179,101.082152,1.159687,0.786007,127.05,81.207857,51.433712,41.639878,6.9055,3.822571,3.479475,1.037439,1.088575,0.927479,12.878,1.757143,4.599064,4.649635,107.756645,60.943760,7.062488,0.619095,0.308148,0.250477,399.97342,57.127669,168.854244,138.540613,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,775.425,1

In [5]:
train.dtypes

number_of_elements         int64
mean_atomic_mass         float64
wtd_mean_atomic_mass     float64
gmean_atomic_mass        float64
wtd_gmean_atomic_mass    float64
                          ...   
range_Valence              int64
wtd_range_Valence        float64
std_Valence              float64
wtd_std_Valence          float64
critical_temp            float64
Length: 82, dtype: object

In [6]:
train.shape

(21263, 82)

In [7]:
unique.head()

,H,He,Li,Be,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,S,Cl,Ar,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Kr,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Xe,Cs,Ba,La,Ce,Pr,Nd,Pm,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.20,1.80,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,29.0,Ba0.2La1.8Cu1O4
1,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.10,1.90,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,26.0,Ba0.1La1.9Ag0.1Cu0.9O4
2,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.10,1.90,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,19.0,Ba0.1La1.9Cu1O4
3,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.15,1.85,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,22.0,Ba0.15La1.85Cu1O4
4,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.30,1.70,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,23.0,Ba0.3La1.7Cu1O4


In [8]:
unique.dtypes

H                float64
He                 int64
Li               float64
Be               float64
B                float64
                  ...   
Po                 int64
At                 int64
Rn                 int64
critical_temp    float64
material          object
Length: 88, dtype: object

In [9]:
unique.shape

(21263, 88)

----
<a id="DataPrep"></a>
<a href="#top">Back to Top</a>
## Data Preperation

<!-- ---- -->
<a id="Scale"></a>
<!-- <a href="#top">Back to Top</a> -->
### Data Scaling
Different scalers can be use

In [10]:
# Standardizing the features using StandardScaler
scaler = StandardScaler()

In [11]:
# Standardizing the features using StandardScaler
# transformer = QuantileTransformer(output_distribution='uniform')

<!-- ---- -->
<a id="Target"></a>
<!-- <a href="#top">Back to Top</a> -->
### Seperate Target

In [12]:
# Prepare training data for models
X = train.drop(['critical_temp'], axis=1) 
y = train['critical_temp']

<!-- ---- -->
<a id="TTSplit"></a>
<!-- <a href="#top">Back to Top</a> -->
### Train/Test Split

In [13]:
# Splitting the data into training and test sets using 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

----
<a id="ModelPrep"></a>
<a href="#top">Back to Top</a>
## Model Preperation

L1 (LASSO)

In [14]:
# Initializing and fitting the Lasso regression model
alpha = 1.0  # Regularization strength
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

c:\Users\bfo2f\.conda\envs\ML1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+06, tolerance: 2.005e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Lasso()

L2 (RIDGE)

In [15]:
# Initializing and fitting the Ridge regression model
alpha = 1.0  # Regularization strength
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X_train, y_train)

Ridge()

----
<a id="Eval"></a>
<a href="#top">Back to Top</a>
## Model Evaluation

L1 (LASSO)

In [16]:
# Making predictions on the test set
y_pred_lasso = lasso_model.predict(X_test)
# Evaluating the Lasso model performance
# Calculate Mean Squared Error (MSE)
mseL = mean_squared_error(y_test, y_pred_lasso)

# Print the evaluation metrics
print(f"MSE: {mseL}")

MSE: 328.0935803528748


L2 (RIDGE)

In [17]:
# Making predictions on the test set
y_pred_ridge = ridge_model.predict(X_test)

# Evaluating the Lasso model performance
# Calculate Mean Squared Error (MSE)
mseR = mean_squared_error(y_test, y_pred_ridge)

# Print the evaluation metrics
print(f"MSE: {mseR}")

MSE: 302.81740743081696


----
<a id="Feat"></a>
<a href="#top">Back to Top</a>
## Discuss Important Features

L1 (LASSO)

In [18]:
# Get the names of the features with non-zero coefficients
selected_features1 = X.columns[lasso_model.coef_ != 0]

# Print the selected features
print(f"Selected Features: {selected_features1}")

Selected Features: Index(['mean_atomic_mass', 'wtd_gmean_atomic_mass', 'range_atomic_mass',
       'wtd_range_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie',
       'wtd_mean_fie', 'gmean_fie', 'wtd_gmean_fie', 'range_fie',
       'wtd_range_fie', 'std_fie', 'wtd_std_fie', 'mean_atomic_radius',
       'wtd_mean_atomic_radius', 'gmean_atomic_radius',
       'wtd_gmean_atomic_radius', 'range_atomic_radius',
       'wtd_range_atomic_radius', 'std_atomic_radius', 'mean_Density',
       'wtd_mean_Density', 'gmean_Density', 'wtd_gmean_Density',
       'range_Density', 'wtd_range_Density', 'std_Density', 'wtd_std_Density',
       'mean_ElectronAffinity', 'wtd_mean_ElectronAffinity',
       'gmean_ElectronAffinity', 'wtd_gmean_ElectronAffinity',
       'range_ElectronAffinity', 'wtd_range_ElectronAffinity',
       'std_ElectronAffinity', 'wtd_std_ElectronAffinity', 'mean_FusionHeat',
       'wtd_gmean_FusionHeat', 'range_FusionHeat', 'wtd_range_FusionHeat',
       'wtd_std_FusionHeat', 'wtd_m

In [19]:
print(f'Coefficients with Lasso: {lasso_model.coef_}')
print(f'Intercept with Lasso: {lasso_model.intercept_}')

Coefficients with Lasso: [-0.00000000e+00  1.46202765e-01 -0.00000000e+00 -0.00000000e+00
 -2.49375473e-02 -0.00000000e+00  0.00000000e+00  2.10317981e-01
 -9.83178356e-02  0.00000000e+00 -3.72673751e-01  6.53366641e-02
  7.04141197e-02 -2.94021910e-02 -8.27202945e-02 -0.00000000e+00
  0.00000000e+00  8.87226177e-02  3.21599165e-02 -2.10571012e-01
 -2.68141456e-02  1.73247824e-02  8.14910356e-01 -1.44045531e-01
 -5.83830181e-01 -0.00000000e+00  0.00000000e+00  1.69765131e-01
 -1.18671896e-01 -3.02498924e-01  0.00000000e+00 -3.56663795e-03
 -2.08409001e-03  1.24765422e-03  2.54270727e-03 -0.00000000e+00
 -0.00000000e+00 -8.37279462e-04  1.21755970e-03  3.19900214e-03
 -1.03176854e-03 -2.64059221e-01  7.31515551e-01  2.17710204e-01
 -7.53033091e-01 -0.00000000e+00 -0.00000000e+00 -3.59918542e-01
 -7.57189564e-02  1.17180832e+00 -5.66462336e-01  9.56956065e-02
  0.00000000e+00  0.00000000e+00  1.25428035e-03  0.00000000e+00
  0.00000000e+00 -4.05550085e-02  1.41834738e-01  0.00000000e+00


L2 (RIDGE)

In [20]:
# Get the names of the features with non-zero coefficients
selected_features2 = X.columns[ridge_model.coef_ != 0]

# Print the selected features
print(f"Selected Features: {selected_features2}")

Selected Features: Index(['number_of_elements', 'mean_atomic_mass', 'wtd_mean_atomic_mass',
       'gmean_atomic_mass', 'wtd_gmean_atomic_mass', 'entropy_atomic_mass',
       'wtd_entropy_atomic_mass', 'range_atomic_mass', 'wtd_range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_mean_fie',
       'gmean_fie', 'wtd_gmean_fie', 'entropy_fie', 'wtd_entropy_fie',
       'range_fie', 'wtd_range_fie', 'std_fie', 'wtd_std_fie',
       'mean_atomic_radius', 'wtd_mean_atomic_radius', 'gmean_atomic_radius',
       'wtd_gmean_atomic_radius', 'entropy_atomic_radius',
       'wtd_entropy_atomic_radius', 'range_atomic_radius',
       'wtd_range_atomic_radius', 'std_atomic_radius', 'wtd_std_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density',
       'wtd_gmean_Density', 'entropy_Density', 'wtd_entropy_Density',
       'range_Density', 'wtd_range_Density', 'std_Density', 'wtd_std_Density',
       'mean_ElectronAffinity', 'wtd_mean_ElectronAffini

In [21]:
print(f'Coefficients with Ridge: {ridge_model.coef_}')
print(f'Intercept with Ridge: {ridge_model.intercept_}')

Coefficients with Ridge: [-3.99566664e+00  8.70564944e-01 -9.23188860e-01 -5.28857799e-01
  6.66920497e-01 -3.15915527e+01  4.07896696e+00  2.22527090e-01
  2.95830176e-02 -5.58724994e-01  7.59255446e-02  6.89825720e-02
  3.76905111e-02 -6.94203794e-02 -1.14466033e-02 -2.17740695e+01
  3.76138656e+01  6.54383963e-02  1.73687121e-02 -1.60379925e-01
 -5.70288501e-02 -2.02239222e-01  2.51540286e+00 -1.14254099e-01
 -2.12750182e+00  1.66784422e+01  3.73319049e+01  2.23325450e-01
 -1.07066666e-01 -6.11559931e-01 -1.41336430e-01 -5.44660673e-03
  5.51786193e-04  1.76931281e-03  1.59607497e-03  1.70171619e+01
 -1.87016549e+01 -1.54093226e-03 -1.36461375e-04  6.46275804e-03
 -1.90592962e-03 -1.50081341e-01  5.59345710e-01  2.21625327e-01
 -6.11144352e-01  3.98411181e+00 -2.15056130e+01 -3.89791682e-01
 -1.54797789e-01  1.31068065e+00 -5.59711706e-01  1.69320509e+00
 -1.93249423e+00 -1.49239973e+00  1.58233455e+00 -1.80073368e+01
  2.52731714e+01 -3.61499859e-01  6.14190440e-01 -5.97549080e-01


In [22]:
selected_features1.shape

(49,)

In [23]:
selected_features2.shape

(81,)

In [24]:
# Find the common features
common_features = set(selected_features1) & set(selected_features2)

# Print the common features
print(len(common_features))
common_features

49


{'gmean_Density',
 'gmean_ElectronAffinity',
 'gmean_ThermalConductivity',
 'gmean_atomic_radius',
 'gmean_fie',
 'mean_Density',
 'mean_ElectronAffinity',
 'mean_FusionHeat',
 'mean_atomic_mass',
 'mean_atomic_radius',
 'mean_fie',
 'range_Density',
 'range_ElectronAffinity',
 'range_FusionHeat',
 'range_ThermalConductivity',
 'range_Valence',
 'range_atomic_mass',
 'range_atomic_radius',
 'range_fie',
 'std_Density',
 'std_ElectronAffinity',
 'std_ThermalConductivity',
 'std_atomic_radius',
 'std_fie',
 'wtd_gmean_Density',
 'wtd_gmean_ElectronAffinity',
 'wtd_gmean_FusionHeat',
 'wtd_gmean_ThermalConductivity',
 'wtd_gmean_atomic_mass',
 'wtd_gmean_atomic_radius',
 'wtd_gmean_fie',
 'wtd_mean_Density',
 'wtd_mean_ElectronAffinity',
 'wtd_mean_ThermalConductivity',
 'wtd_mean_atomic_radius',
 'wtd_mean_fie',
 'wtd_range_Density',
 'wtd_range_ElectronAffinity',
 'wtd_range_FusionHeat',
 'wtd_range_ThermalConductivity',
 'wtd_range_atomic_mass',
 'wtd_range_atomic_radius',
 'wtd_range_

----
<a id="Cite"></a>
<a href="#top">Back to Top</a>
## Citations